In [67]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"],  # modules to install
    license_uuid="bb2c71f6-2b4c-4570-8eaf-4db0b0d7349a", )# su UUID de licencia


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\gabop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Licensed to Bundle #6733.7184 expiring 20241231: ICI5142-01 INVESTIGACION DE OPERACIONES AVANZADAS (Advanced Operations Research), Prof. Guillermo Cabrera-Guerrero, Pontificia Universidad Catolica de Valparaiso.


In [68]:
%%writefile sscflp.mod
# Definición de parámetros
param n; # Número de clientes
param m; # Número de centros de distribución
param c{i in 1..n, j in 1..m}; # Costo de transporte del cliente i al centro j
param f{j in 1..m}; # Costo fijo de apertura del centro j
param d{i in 1..n}; # Demanda del cliente i
param b{j in 1..m}; # Capacidad del centro j

# Definición de variables de decisión
var x{j in 1..m} binary; # Si el centro j está abierto
var y{i in 1..n, j in 1..m} binary; # Si el cliente i es atendido por el centro j

# Función objetivo: Minimizar los costos totales
minimize TotalCost: sum{i in 1..n, j in 1..m} c[i,j] * y[i,j] + sum{j in 1..m} f[j] * x[j];

# Restricción: Cada cliente debe ser atendido por exactamente un centro
subject to AssignCliente{i in 1..n}: sum{j in 1..m} y[i,j] = 1;

# Restricción: La capacidad del centro no debe ser excedida
subject to Capacity{j in 1..m}: sum{i in 1..n} d[i] * y[i,j] <= b[j] * x[j];

# Restricción: Un cliente solo puede ser asignado a un centro si el centro está abierto
subject to OpenIfAssigned{i in 1..n, j in 1..m}: y[i,j] <= x[j];

Overwriting sscflp.mod


In [69]:
%%writefile mscflp.mod 
# Definición de parámetros
param m; # Número de centros de distribución
param n; # Número de clientes
param f{i in 1..m}; # Costo de apertura del centro i
param c{i in 1..m, j in 1..n}; # Costo de transporte del centro i al cliente j
param S{i in 1..m}; # Capacidad del centro i
param D{j in 1..n}; # Demanda del cliente j

# Definición de variables de decisión
var x{i in 1..m, j in 1..n} >= 0; # Cantidad de demanda del cliente j atendida por el centro i
var y{i in 1..m} binary; # Si el centro i está abierto

# Función objetivo: Minimizar los costos totales
minimize TotalCost: sum{i in 1..m} f[i] * y[i] + sum{i in 1..m, j in 1..n} c[i,j] * x[i,j];

# Restricción: La capacidad del centro no debe ser excedida
subject to Capacity{i in 1..m}: sum{j in 1..n} x[i,j] <= S[i];

# Restricción: La demanda de cada cliente debe ser satisfecha
subject to Demand{j in 1..n}: sum{i in 1..m} x[i,j] >= D[j];

# Restricción: Un centro solo puede satisfacer demanda si está abierto
subject to OpenIfAssigned{i in 1..m, j in 1..n}: x[i,j] <= S[i] * y[i];


Overwriting mscflp.mod


In [70]:
import os
from amplpy import AMPL
import random
import pandas as pd

def parse_instance(file_path):
    with open(file_path, 'r') as f:
        # Leer primera línea: m (ubicaciones) y n (clientes)
        m, n = map(int, f.readline().split())
        
        # Leer capacidades y costos fijos de cada ubicación
        capacities, fixed_costs = [], []
        for _ in range(m):
            cap, cost = map(float, f.readline().split())  # Usamos float para manejar decimales
            capacities.append(cap)
            fixed_costs.append(cost)
        
        # Leer demandas de clientes y costos de asignación
        demands, allocation_costs = [], []
        for _ in range(n):
            data = list(map(float, f.readline().split()))  # También se usa float aquí
            demand = data[0]
            costs = data[1:]
            demands.append(demand)
            allocation_costs.append(costs)
    
    return m, n, capacities, fixed_costs, demands, allocation_costs


def write_ampl_dat(file_path, m, n, capacities, fixed_costs, demands, allocation_costs):
    with open(file_path, 'w') as f:
        f.write(f'param m := {m};\n')
        f.write(f'param n := {n};\n')
        
        # Capacidad de cada centro
        f.write('param b := ')
        f.write(' '.join(f'{i+1} {cap:.1f}' for i, cap in enumerate(capacities)) + ';\n')
        
        # Costo fijo de cada centro
        f.write('param f := ')
        f.write(' '.join(f'{i+1} {cost:.1f}' for i, cost in enumerate(fixed_costs)) + ';\n')
        
        # Demanda de cada cliente
        f.write('param d := ')
        f.write(' '.join(f'{j+1} {dem:.1f}' for j, dem in enumerate(demands)) + ';\n')
        
        # Completar costos de asignación
        f.write('param c : ' + ' '.join(str(i+1) for i in range(m)) + ' :=\n')
        for j in range(n):
            row_costs = allocation_costs[j] if j < len(allocation_costs) else [0.0] * m
            # Si la fila es incompleta, rellena con ceros
            if len(row_costs) < m:
                row_costs.extend([0.0] * (m - len(row_costs)))
            f.write(f"{j+1} " + ' '.join(f"{cost:.1f}" for cost in row_costs) + '\n')
        f.write(';\n')



def solve_with_ampl(model_file, data_file):
    ampl = AMPL()
    ampl.read(model_file)
    ampl.read_data(data_file)
    ampl.solve()
    total_cost = ampl.get_objective("TotalCost").value()
    return total_cost

# Heurísticas de búsqueda local y Benders
def local_search_sscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100):
    best_solution, best_cost = None, float('inf')
    for _ in range(max_iterations):
        solution = generate_random_solution(m, n, capacities, demands)
        solution_cost = calculate_solution_cost(solution, fixed_costs, allocation_costs)
        if solution_cost < best_cost:
            best_solution, best_cost = solution, solution_cost
        solution = perturb_solution(solution)
    return best_solution, best_cost

def local_search_mscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100):
    best_solution, best_cost = None, float('inf')
    for _ in range(max_iterations):
        solution = generate_random_solution(m, n, capacities, demands)
        solution_cost = calculate_solution_cost(solution, fixed_costs, allocation_costs)
        if solution_cost < best_cost:
            best_solution, best_cost = solution, solution_cost
        solution = perturb_solution(solution)
    return best_solution, best_cost

def benders_decomposition_sscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100):
    best_solution, best_cost = None, float('inf')
    for _ in range(max_iterations):
        solution_cost = benders_iteration() 
        if solution_cost < best_cost:
            best_solution, best_cost = solution, solution_cost
        if convergence_criteria_met():
            break
    return best_solution, best_cost

def benders_decomposition_mscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100):
    best_solution, best_cost = None, float('inf')
    for _ in range(max_iterations):
        solution_cost = benders_iteration()
        if solution_cost < best_cost:
            best_solution, best_cost = solution, solution_cost
        if convergence_criteria_met():
            break
    return best_solution, best_cost


In [71]:
import random

def generate_random_solution(m, n, capacities, demands):
    # Inicializar la solución con asignaciones vacías para cada cliente
    solution = [-1] * n
    remaining_capacity = capacities[:]  # Copia de las capacidades para rastrear el espacio restante

    for j in range(n):
        # Intento de asignar cliente j a un centro aleatorio que tenga capacidad suficiente
        possible_centers = [i for i in range(m) if remaining_capacity[i] >= demands[j]]
        if possible_centers:
            selected_center = random.choice(possible_centers)
            solution[j] = selected_center
            remaining_capacity[selected_center] -= demands[j]
        else:
            # Si no hay centros con capacidad, se asigna -1 (sin asignación válida)
            solution[j] = -1

    return solution


In [72]:
def calculate_solution_cost(solution, fixed_costs, allocation_costs):
    total_cost = 0
    used_centers = set(solution)  # Centros utilizados en la solución

    # Sumar costos fijos de los centros abiertos
    for center in used_centers:
        if center != -1:
            total_cost += fixed_costs[center]

    # Sumar costos de asignación para cada cliente
    for j, center in enumerate(solution):
        if center != -1:
            total_cost += allocation_costs[j][center]

    return total_cost


In [ ]:
# Ruta de las instancias
instance_dir = "Problem set OR Library"

# Selección de instancia (descomentar una línea para elegir)
instance_file = f"{instance_dir}/cap41.txt"
# instance_file = f"{instance_dir}/capa.txt"
# instance_file = f"{instance_dir}/5000x5000.txt"

# Nombre de la instancia para usar en archivos de salida
instance_name = os.path.basename(instance_file).split('.')[0]


In [74]:
def local_search_sscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100):
    best_solution, best_cost = None, float('inf')
    
    for _ in range(max_iterations):
        # Generar una solución inicial aleatoria
        solution = generate_random_solution(m, n, capacities, demands)
        
        # Calcular el costo de la solución inicial
        solution_cost = calculate_solution_cost(solution, fixed_costs, allocation_costs)
        
        # Si es la mejor solución encontrada hasta ahora, actualizar
        if solution_cost < best_cost:
            best_solution, best_cost = solution, solution_cost

        # Aquí puedes implementar la lógica de perturbación (mejora de la solución)
        # Ejemplo: perturbar o intercambiar asignaciones para buscar mejores costos
        solution = perturb_solution(solution)  # Función de perturbación

    return best_solution, best_cost

In [75]:
def benders_iteration():
    # Configuración inicial del subproblema
    # Aquí deberás implementar el subproblema con los cortes de Benders
    # Este es solo un esqueleto para la función
    subproblem_cost = random.uniform(50000, 100000)  # Placeholder: simula un valor de costo
    return subproblem_cost


In [76]:
def perturb_solution(solution):
    # Ejemplo básico de perturbación: intercambiar asignaciones de dos clientes
    new_solution = solution[:]
    j1, j2 = random.sample(range(len(solution)), 2)  # Elegir dos clientes aleatoriamente
    new_solution[j1], new_solution[j2] = new_solution[j2], new_solution[j1]
    return new_solution


In [77]:
#"""
# === Resolución del modelo SSCFLP para la instancia seleccionada ===


# Archivo de datos SSCFLP
sscflp_data_file = f"resultadosInstancias/instance_sscflp_{instance_name}.dat"
m, n, capacities, fixed_costs, demands, allocation_costs = parse_instance(instance_file)
write_ampl_dat(sscflp_data_file, m, n, capacities, fixed_costs, demands, allocation_costs)

# Solución exacta con AMPL
optimal_cost_sscflp = solve_with_ampl("sscflp.mod", sscflp_data_file)
print(f"SSCFLP - Costo óptimo con AMPL para {instance_name}: {optimal_cost_sscflp}")

# Solución heurística con búsqueda local
_, local_search_cost_sscflp = local_search_sscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100)
print(f"SSCFLP - Costo búsqueda local para {instance_name}: {local_search_cost_sscflp}")

# Solución con descomposición de Benders
_, benders_cost_sscflp = benders_decomposition_sscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100)
print(f"SSCFLP - Costo descomposición de Benders para {instance_name}: {benders_cost_sscflp}")

#"""


	presolve, constraint AssignCliente[50]:
		all variables eliminated, but lower bound = 1 > 0
	presolve, constraint AssignCliente[48]:
		all variables eliminated, but lower bound = 1 > 0
	presolve, constraint AssignCliente[47]:
		all variables eliminated, but lower bound = 1 > 0
	presolve, constraint AssignCliente[46]:
		all variables eliminated, but lower bound = 1 > 0
	presolve, constraint AssignCliente[44]:
		all variables eliminated, but lower bound = 1 > 0
	20 presolve messages suppressed.
SSCFLP - Costo óptimo con AMPL para cap41: 0.0
SSCFLP - Costo búsqueda local para cap41: 92173.0875


NameError: name 'solution' is not defined

In [ ]:
"""
# === Resolución del modelo MSCFLP para la instancia seleccionada ===

# Archivo de datos MSCFLP
mscflp_data_file = f"resultadosInstancias/instance_mscflp_{instance_name}.dat"
m, n, capacities, fixed_costs, demands, allocation_costs = parse_instance(instance_file)
write_ampl_dat(mscflp_data_file, m, n, capacities, fixed_costs, demands, allocation_costs)

# Solución exacta con AMPL
optimal_cost_mscflp = solve_with_ampl("mscflp.mod", mscflp_data_file)
print(f"MSCFLP - Costo óptimo con AMPL para {instance_name}: {optimal_cost_mscflp}")

# Solución heurística con búsqueda local
_, local_search_cost_mscflp = local_search_mscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100)
print(f"MSCFLP - Costo búsqueda local para {instance_name}: {local_search_cost_mscflp}")

# Solución con descomposición de Benders
_, benders_cost_mscflp = benders_decomposition_mscflp(m, n, capacities, fixed_costs, demands, allocation_costs, max_iterations=100)
print(f"MSCFLP - Costo descomposición de Benders para {instance_name}: {benders_cost_mscflp}")

"""


In [ ]:
def save_results_to_excel(results, output_dir="resultadosInstancias"):
    os.makedirs(output_dir, exist_ok=True)
    results_file = os.path.join(output_dir, "results.xlsx")
    df = pd.DataFrame(results, columns=[
        "Instance", 
        "SSCFLP Optimal Cost (AMPL)", "SSCFLP Local Search Cost", "SSCFLP Benders Cost",
        #"MSCFLP Optimal Cost (AMPL)", "MSCFLP Local Search Cost", "MSCFLP Benders Cost"
    ])
    df.to_excel(results_file, index=False)
